In [11]:
import os
import copy
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.io
import matplotlib.pyplot as plt
import glob
import re
import sklearn
from datetime import datetime
from dateutil import parser
import dateutil.parser
from scipy.io import savemat

import numpy as np
import scipy.misc
import numpy.random as rng
from PIL import Image, ImageDraw, ImageFont
from sklearn.utils import shuffle
import nibabel as nib #reading MR images
import math
from matplotlib import pyplot as plt
from scipy.stats import pearsonr

from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from operator import itemgetter

In [12]:
DATASET = 'Dataset3.0.csv'
dataset = pd.read_csv(DATASET)
dataset['Sex'] = dataset['Sex'].replace({'M': 1, 'F': 0})
lipids_only = dataset.iloc[:, list(range(54, 837))]
columns_to_drop = ['LHC_ICV', 'RHC_ICV']
X = lipids_only.drop(columns=columns_to_drop, axis=1)
y = lipids_only['LHC_ICV']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

estimator = RandomForestRegressor(n_estimators=100, random_state=42)  # You can adjust parameters
selector = RFE(estimator, n_features_to_select=20, step=1)
selector = selector.fit(X_train, y_train)

y_pred = selector.predict(X_test)

def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

mape = MAPE(y_test, y_pred)
print(f"Mean Percentage Error on Test Set: {mape:.4f}%")

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R-squared:", r2)

print("Actual vs Predicted Values:")
for i in range(4):
    print(f"Example {i + 1}: Actual = {y_test.iloc[i]}, Predicted = {y_pred[i]}")

features = X_train.columns.to_list()
for x, y in (sorted(zip(selector.ranking_ , features), key=itemgetter(0))):
    print(x, y)

selected_features = X.columns[selector.support_]
print("Selected Features:")
print(selected_features)

Mean Percentage Error on Test Set: 15.9179%
Mean Squared Error: 1.5553538167739456e-07
Mean Absolute Error: 0.0003153206175411765
R-squared: 0.20776985164550932
Actual vs Predicted Values:
Example 1: Actual = 0.001567206, Predicted = 0.00184611603
Example 2: Actual = 0.00144, Predicted = 0.00179328191
Example 3: Actual = 0.002216442, Predicted = 0.0025806976000000005
Example 4: Actual = 0.002247962, Predicted = 0.00209316162
1 SPH.D18.2.
1 HEX2CER.D16.1.16.0.
1 GM3.D18.1.20.0.
1 GM3.D18.1.24.1.
1 SM.D18.0.22.0.
1 SM.D18.2.23.0.
1 LPC.18.1...SN1.
1 LPC.O.24.1.
1 PE.P.16.0.22.6.
1 PE.P.18.1.18.1...A.
1 LPE.18.2...SN1.
1 LPE.20.4...SN1.
1 LPE.22.6...SN2.
1 DE.18.2.
1 DEDE.18.2.
1 DEDE.20.4.
1 FA.17.1.
1 AC.18.0.
1 AC.18.2.
1 TG.52.4...NL.16.1.
2 TG.O.50.1...NL.18.1.
3 TG.54.3...NL.18.2.
4 S1P.D18.2.
5 AC.14.0.
6 DE.20.4.
7 LPC.18.2....OH.
8 METHYL.DE.18.1.
9 SM.41.0.
10 PC.18.0_20.3.
11 DHCER.D18.0.16.0.
12 AC.18.1.
13 CER.D19.1.16.0.
14 TG.52.3...SIM.
15 SM.D18.1.23.0..SM.D17.1.24.0.
16 